## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
#импортировали библиотеки
import pandas as pd
import numpy as np
from pymystem3 import Mystem
m = Mystem()
from collections import Counter

#прочитали данные из файла
data=pd.read_csv('/datasets/data.csv')

In [2]:
#вывели информацию о данных
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
#вывели данные на экран
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


### Вывод

В полученном файле 21525 строк и 12 столбцов. В столбцах children и days_employed есть отрицательные значение, чего быть не может. В столбцах days_employed и total_income по 2174 пустых значений. В столбце dob_years есть возраст, равный 0. В столбце education одни и те же данные записаны в разном регистре. В столбце family_status все данные с маленькой буквы, кроме одного. В столбце gender есть одна строка с неизвечтным полом. В столбце purpose наблюдаются одинаковые цели, записаные разными словами.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [4]:
#нашли количество пропусков в столбце days_employed
print(data[data['days_employed'].isnull()].count())

children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64


In [5]:
#избавились от отрицательных значений в столбце days_employed
data['days_employed']=data['days_employed'].abs()
#в переменную days_employed_avg сохранили среднее значение по столбцу days_employed
days_employed_avg=data['days_employed'].mean()
#заменили пустые значения в толбце days_employed на среднее значение
data['days_employed']=data['days_employed'].fillna(value=days_employed_avg)

In [6]:
#нашли количество пропусков в столбце total_income
print(data[data['total_income'].isnull()].count())

children            2174
days_employed       2174
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64


In [7]:
#в переменную total_income_avg сохранили среднее значение по столбцу total_income
total_income_avg=data['total_income'].mean()
#заменили пустые значения в толбце total_income на среднее значение
data['total_income']=data['total_income'].fillna(value=total_income_avg)

In [8]:
#в переменную dob_years_avg сохранили среднее значение по столбцу dob_years
dob_years_avg=data['dob_years'].mean()
#заменили нулевые значения в толбце dob_years на округленное среднее значение
#(так как остальные значение возраста по столбцу целые)
data['dob_years'] = data['dob_years'].replace(0,round(dob_years_avg))

In [9]:
#заменили неизвестны пол в столбце gender на женский. без разницы на какой пол заменять, 
#потому что количество строк с неизвестным полом много меньше и мужского и женского
data.loc[data['gender'] == 'XNA', 'gender'] = 'F'

In [10]:
#избавились от отрицательных значений в столбце children
data['children']=data['children'].abs()
#заменили значения, равные 4,5 и 20 на 3, 
#так как можно объединить количество детей больше или равное 3 в из-за того, 
#что эти значения по этим строкам очень малы
data['children'] = data['children'].replace(20,3)
data['children'] = data['children'].replace(4,3)
data['children'] = data['children'].replace(5,3)
#data['children'].value_counts()

### Вывод

В столбце days_employed отрицательные значение были заменены на положительные. В столбце total_income отрицальных строк нет. Пустые строки в столбцах days_employed и total_income были замененны на среднее значение по столбцу. В столбце dob_years нулевое значение возраста было заменено на среднее значение по столбцу. Неопределенный пол в столбце gender был заменен на женский. Отрицательные значения в столбце children были заменены на положительные, также количество детей больше 3 были объединены в строку 3, в связи с тем что значения по таким строкам были много меньше значений по строкам 0,1,2.


Причина пустых значений может быть связанна с тем, что клиент не заполнил эти поля при подаче документов на оформление кредита.
Причина отрицательных значений в столбце days_employed может быть связана с методикой расчета.
Наличие нулевого значения возраста может быть связано с ошибкой заполнений данных.
Наличие неизвестного пола в столбце gender также может быть связана с ошибкой заполнений данных.
Отрицательные значений и аномально большое значение 20 в столбце children  может быть связана с ошибкой заполнений данных.

### Замена типа данных

In [11]:
#заменили тип данных у столбцов days_employed и total_income
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
#data.info()

### Вывод

Вещественный тип в столбцах days_employed и total_income был заменен на целочисленный методом astype.

### Обработка дубликатов

In [12]:
#привели столбец education к одному регистру, чтобы убрать одни и те же названия в разном регистре
data['education'] = data['education'].str.lower()
#привели столбец family_status к одному регистру для красоты
data['family_status'] = data['family_status'].str.lower()

In [13]:
#нашли количество дубликатов
data.duplicated().sum()
#вывели их на экран
data[data.duplicated()==True]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,66914,41,среднее,1,женат / замужем,0,F,сотрудник,0,167422,покупка жилья для семьи
3290,0,66914,58,среднее,1,гражданский брак,1,F,пенсионер,0,167422,сыграть свадьбу
4182,1,66914,34,высшее,0,гражданский брак,1,F,сотрудник,0,167422,свадьба
4851,0,66914,60,среднее,1,гражданский брак,1,F,пенсионер,0,167422,свадьба
5557,0,66914,58,среднее,1,гражданский брак,1,F,пенсионер,0,167422,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,66914,64,среднее,1,женат / замужем,0,F,пенсионер,0,167422,дополнительное образование
21032,0,66914,60,среднее,1,женат / замужем,0,F,пенсионер,0,167422,заняться образованием
21132,0,66914,47,среднее,1,женат / замужем,0,F,сотрудник,0,167422,ремонт жилью
21281,1,66914,30,высшее,0,женат / замужем,0,F,сотрудник,0,167422,покупка коммерческой недвижимости


In [14]:
#удалили дубликаты
data=data.drop_duplicates()
#вывели получившийся датасет на экран
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21521,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


### Вывод

К столбцу education был примене метод str.lower для устранения одинаковых названий образования, записанных в разном регистре. К столбцу family_status был применен этот же метод для того, чтобы все названия семейного положения были записаны единообразно. После этого методом duplicated() был найден 71 дубль, которые были удалены методом drop_duplicates(). Количество строк в таблице стало равно 21454.

Причина одинаковых названий образования может заключаться в отсутствии единой методики при заполнении анкет клиентами банка. Аналогично, и для столбца семейное положение. Причина наличия дубликатов в данных может быть связана с некорректностью переноса сотрудниками банка данных о клиентах в систему.

### Лемматизация

In [15]:
#вывели причины для кредитования и количество заявок по каждой причине
data['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [16]:
#записали в переменную purpose_unique уникальные значения столбца purpose
purpose_unique=data['purpose'].unique()
#записали все цели через пробел
str_purpose_unique=' '.join(purpose_unique)
#разбили цели на леммы
lemmas=m.lemmatize(str_purpose_unique)
print(lemmas)

['покупка', ' ', 'жилье', ' ', 'приобретение', ' ', 'автомобиль', ' ', 'дополнительный', ' ', 'образование', ' ', 'сыграть', ' ', 'свадьба', ' ', 'операция', ' ', 'с', ' ', 'жилье', ' ', 'образование', ' ', 'на', ' ', 'проведение', ' ', 'свадьба', ' ', 'покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', ' ', 'покупка', ' ', 'недвижимость', ' ', 'покупка', ' ', 'коммерческий', ' ', 'недвижимость', ' ', 'покупка', ' ', 'жилой', ' ', 'недвижимость', ' ', 'строительство', ' ', 'собственный', ' ', 'недвижимость', ' ', 'недвижимость', ' ', 'строительство', ' ', 'недвижимость', ' ', 'на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', ' ', 'на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', ' ', 'операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', ' ', 'строительство', ' ', 'жилой', ' ', 'недвижимость', ' ', 'жилье', ' ', 'операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', ' ', 'автомобиль', ' ', 'заниматься', ' ', 'образование', ' ', 'сделка', ' ', 'с', ' ', 'подержать

In [17]:
#посчитали количество лемм
print(Counter(lemmas))

Counter({' ': 96, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, '\n': 1})


In [18]:
#создадим функцию, для разбиения целей взятия кредита по группам
def purpose_group(purpose):
        if ('свадьба' in purpose or 'свадьбы'in purpose or 'свадьбу'in purpose):
                return 'свадьба'
        if ("автомобиля" in purpose or "автомобилем" in purpose or "автомобили" in purpose or "автомобиль" in purpose):
                return 'автомобиль'
        if ("образование" in purpose or "образованием" in purpose or "образования" in purpose):
                return 'образование'
        if ("недвижимости" in purpose or "недвижимость" in purpose 
            or "недвижимостью" in purpose or "жилье" in purpose
            or "жилью" in purpose
           or "жилья" in purpose):
                return 'недвижимость'
        return purpose
#добавим столбец в исходный датасет
data['purpose_group']=data['purpose'].apply(purpose_group)
print(data)

       children  days_employed  dob_years education  education_id  \
0             1           8437         42    высшее             0   
1             1           4024         36   среднее             1   
2             0           5623         33   среднее             1   
3             3           4124         32   среднее             1   
4             0         340266         53   среднее             1   
...         ...            ...        ...       ...           ...   
21520         1           4529         43   среднее             1   
21521         0         343937         67   среднее             1   
21522         1           2113         38   среднее             1   
21523         3           3112         38   среднее             1   
21524         2           1984         40   среднее             1   

          family_status  family_status_id gender income_type  debt  \
0       женат / замужем                 0      F   сотрудник     0   
1       женат / замужем        

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [19]:
data['purpose_group'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_group, dtype: int64

### Вывод

Была проведена лемматизация данных по столбцу purpose методом lemmatize. Выявлены основные причины для того, чтобы взять кредит: операции с недвижимостью, покупка автомобиля, образование и свадьба.

### Категоризация данных

In [20]:
#вывели максимальное и минимальное возрасты
print(data['dob_years'].min())
print(data['dob_years'].max())

19
75


In [21]:
#создадим функцию, для разбиения возраста по группам
def age_group(dob_years):
        if (dob_years>=19 and dob_years<= 24):
                return '19-25'
        if (dob_years>= 25 and dob_years<= 35):
                return '25-35'
        if (dob_years>= 36 and dob_years<= 55):
                return '36-55'
        if dob_years>= 56:
                return '56+'
#добавим столбец с разбиением по возрастам в исходный датасет
data['age_group']=data['dob_years'].apply(age_group)
print(data)

       children  days_employed  dob_years education  education_id  \
0             1           8437         42    высшее             0   
1             1           4024         36   среднее             1   
2             0           5623         33   среднее             1   
3             3           4124         32   среднее             1   
4             0         340266         53   среднее             1   
...         ...            ...        ...       ...           ...   
21520         1           4529         43   среднее             1   
21521         0         343937         67   среднее             1   
21522         1           2113         38   среднее             1   
21523         3           3112         38   среднее             1   
21524         2           1984         40   среднее             1   

          family_status  family_status_id gender income_type  debt  \
0       женат / замужем                 0      F   сотрудник     0   
1       женат / замужем        

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [22]:
data['total_income'].median()

156043.5

In [23]:
data['total_income'].min()

20667

In [24]:
data['total_income'].max()

2265604

In [25]:
#создадим функцию, для разбиения по уровню дохода
def total_income_group(total_income):
        if (total_income>=20000 and total_income<= 75000):
                return '20-75'
        if (total_income>= 76000 and total_income<= 150000):
                return '76-150'
        if (total_income>= 151000 and total_income<= 300000):
                return '151-300'
        if total_income>= 301000:
                return '301+'
#добавим столбец с разбиением в исходный датасет
data['total_income_group']=data['total_income'].apply(total_income_group)
print(data)

       children  days_employed  dob_years education  education_id  \
0             1           8437         42    высшее             0   
1             1           4024         36   среднее             1   
2             0           5623         33   среднее             1   
3             3           4124         32   среднее             1   
4             0         340266         53   среднее             1   
...         ...            ...        ...       ...           ...   
21520         1           4529         43   среднее             1   
21521         0         343937         67   среднее             1   
21522         1           2113         38   среднее             1   
21523         3           3112         38   среднее             1   
21524         2           1984         40   среднее             1   

          family_status  family_status_id gender income_type  debt  \
0       женат / замужем                 0      F   сотрудник     0   
1       женат / замужем        

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [26]:
#создали словарь образования
education_dict = data[['education_id', 'education']]
print(education_dict.head(10))

   education_id education
0             0    высшее
1             1   среднее
2             1   среднее
3             1   среднее
4             1   среднее
5             0    высшее
6             0    высшее
7             1   среднее
8             0    высшее
9             1   среднее


In [27]:
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
print(education_dict)

   education_id            education
0             0               высшее
1             1              среднее
2             2  неоконченное высшее
3             3            начальное
4             4       ученая степень


In [28]:
#создали словарь семейного положения
family_status_dict = data[['family_status_id', 'family_status']]
print(family_status_dict.head(10))

   family_status_id     family_status
0                 0   женат / замужем
1                 0   женат / замужем
2                 0   женат / замужем
3                 0   женат / замужем
4                 1  гражданский брак
5                 1  гражданский брак
6                 0   женат / замужем
7                 0   женат / замужем
8                 1  гражданский брак
9                 0   женат / замужем


In [29]:
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
print(family_status_dict)

   family_status_id          family_status
0                 0        женат / замужем
1                 1       гражданский брак
2                 2         вдовец / вдова
3                 3              в разводе
4                 4  не женат / не замужем


### Вывод

Были созданы два словаря education_dict, family_status_dict и добавлены новые столбецы с разбиением по возрасту и доходу. Столбец разбиения по доходу был выбран изходя из медианы. Взяла две группы выше и ниже медианы.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [30]:
data_pivot = data.pivot_table(index = ['family_status'], columns = 'children',values = 'debt', aggfunc = ['mean'])
data_pivot.style.format('{:.3%}')

### Вывод

Да, есть зависимость между наличием детей и возвратом кредита в срок, а также между семейным положением и возвратом кредита в срок. Самый большой прцоент задолженности у не женатых/ не замужних людей. Самый маленький у вдовцов/вдов. Самый большой прцоент задолженности у людей с 2 детьми, самый маленький - у бездетных.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [31]:
data_pivot_income=data.pivot_table(index=['total_income_group'], columns='debt',values='total_income',aggfunc='sum')
data_pivot_income['ratio']=data_pivot_income[0]/(data_pivot_income[0]+data_pivot_income[1])*100
print(data_pivot_income)

debt                         0          1      ratio
total_income_group                                  
151-300             1762873115  150399857  92.139132
20-75                103249563    8307697  92.552975
301+                 567086056   43263415  92.911698
76-150               854090993   80085548  91.427151


### Вывод

Отношение отсутствия задолженности по возврату кредитов ко всем кредитам представлено в столбце ratio. Это отношение почти одинаково по всем группам дохода. Можно сделать вывод, что на полученных данных доход не влияет на возвращение кредита в срок.

- Как разные цели кредита влияют на его возврат в срок?

In [32]:
#data_pivot_purpose['ratio']=data_pivot_purpose[0]/(data_pivot_purpose[1]+data_pivot_purpose[0])
#print(data_pivot_purpose)
data_pivot_purpose=data.pivot_table(index=['purpose_group'], columns='total_income_group',values='debt',aggfunc=['mean'])
data_pivot_purpose.style.format('{:.3%}')

### Вывод

Самые большие прцоенты задолженности на автомобиль и образование. Самый маленький - на недвижимость.

### Шаг 4. Общий вывод

Был проведен анализ банка о статистика о платёжеспособности клиентов. На вход был получен датасет с 12 столбцами и 21525 строками. 

В данных были обнаружены пропуски и дубликаты. Причиной наличия дубликатов и пропусков может быть некорректно заполненные анкеты или неверный перенос данных в систему. Дубликаты и пропуски были устранены. Значения столбца количество детей были объединены в группы: 0,1,2 и 3+. После устранения, строк в датасете стало 21454. 

Затем была проведена лемматизация по столбцу причин взятия кредита. Были выделены основные цели взятия кредита, а именно: образование, свадьба, недвижимость и автомобиль. Также были выделенны возрастные группы заемщиков:'19-25','25-35','36-55' и'56+'; а доход был разбит по группам выше и ниже медианы:'20-75', '76-150', '151-300' и '301+'. 

Затем была проведена категоризация данных и созданы словари образования и семейного положения. 

После чего были выявлены зависимости влияния наличия детей, семейного положения и уровня дохода на возврат кредита в срок, а также как цели кредита влияют на это.